In [1]:
import pandas as pd
import spacy

2022-12-25 19:03:57.868746: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-25 19:03:58.438578: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.4/lib64
2022-12-25 19:03:58.438619: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.4/lib64
2022-12-25 19:03:58.438624: W tensorflow/compiler/tf2tensorrt/utils/py_utils

### Load data

In [2]:
lang = 'zh'
Set = 'train'
extractor = 'all'

In [3]:
data= pd.read_csv(f'../Dataset/{Set}.csv')
data.lang.unique()

array(['bn', 'de', 'en', 'es', 'fa', 'fr', 'hi', 'it', 'pt', 'sv', 'uk',
       'zh'], dtype=object)

In [4]:
data = data[data['lang']==lang]

### Information Extraction Pipeline

In [5]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("Davlan/xlm-roberta-base-ner-hrl")
model = AutoModelForTokenClassification.from_pretrained("Davlan/xlm-roberta-base-ner-hrl")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)


In [6]:
# from transformers import AutoTokenizer, AutoModelForTokenClassification
# from transformers import pipeline
# tokenizer = AutoTokenizer.from_pretrained("jplu/tf-xlm-r-ner-40-lang")
# model = AutoModelForTokenClassification.from_pretrained("jplu/tf-xlm-r-ner-40-lang",from_tf=True)
# nlp = pipeline("ner", model=model, tokenizer=tokenizer)

In [7]:
#import the pipeline class


from InformationExtraction import InformationExtractionPipeline


# example spacy extractor function
NER = spacy.load("en_core_web_lg")
def tag_extraction_from_spacy(sen, model = NER):
    
    annotated = model(sen)
    extracted_names = [word.text for word in annotated.ents 
                       if word.label_=='PERSON' or word.label_=='ORG'or word.label_=='GPE']
    
    
    return extracted_names


def tag_extraction_from_LM(sen, model = nlp):
    
    ner_results = model(sen)
    extracted_names = []
    for idx in range(len(ner_results)):
        if ner_results[idx]['entity'][0] == 'B':
            start = ner_results[idx]['start']
            end = ner_results[idx]['end']
            j = idx+1
            while j < len(ner_results):
                if ner_results[j]['entity'][0] == 'B':
                    break
                elif ner_results[j]['entity'][0] == 'I':
                    end = ner_results[j]['end']
                j+=1
            idx = j
        
            extracted_names.append(sen[start:end].strip())
    
    
    return extracted_names


# example extractor function that uses training labels 
sent_to_tag = dict(zip(data['sent'],data['labels']))
def tag_extraction_from_tags(sent, sent_to_tag=sent_to_tag):

    tags = sent_to_tag[sent]
    sentsWithtags = [(s,t) for s,t in zip(sent.split(),tags.split())]
    entity_list = []
    for i,item in enumerate(sentsWithtags):
        if 'B-' in item[1]:
            j = i
            entity = []
            while j<len(sentsWithtags):
                if sentsWithtags[j][1] =='O':
                    break
                entity.append(sentsWithtags[j][0])
                j+=1
            i = j
         
            entity_list.append(" ".join(entity))
            
    

    return entity_list




In [8]:
tag_extraction_from_LM('নিউটন একজন বিজ্ঞানী')

['নিউটন']

In [9]:
#create pipline object:
#param: extractor: an entity extractor function that returns all the entities from a sentence
#param: max_sen: define the number of sentences to be added for each detected entity
#param: lang: define language. needed for wikipedia api
#param: saveJson: whether to save extracted informaton as json file. Saves time if needed to run the pipeline again
#param: loadJson: if you have saved a json file and want to use it
#param: jsonPath: define saved json file path


infoPipeline = InformationExtractionPipeline(extractor = tag_extraction_from_tags if extractor=='all' else tag_extraction_from_LM, 
                                        max_sen = 2, lang = lang, 
                                        loadJson = False, jsonPath=f'wiki-info-{lang}-{Set}.json',
                                        saveJson=True, saveJsonpath=f'wiki-info-{lang}-{Set}.json')

In [10]:
#call pipline and provide list of sentences as argument

augmented = infoPipeline(data['sent'].values.tolist())

  6%|████████                                                                                                                                    | 564/9758 [02:42<41:53,  3.66it/s]

HTTPSConnectionPool(host='zh.wikipedia.org', port=443): Read timed out. (read timeout=10.0)


 13%|█████████████████▍                                                                                                                         | 1224/9758 [07:58<39:12,  3.63it/s]

HTTPSConnectionPool(host='zh.wikipedia.org', port=443): Read timed out. (read timeout=10.0)


 16%|██████████████████████▊                                                                                                                    | 1598/9758 [12:02<44:28,  3.06it/s]

...

 82%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                         | 7959/9758 [45:04<10:39,  2.82it/s]

HTTPSConnectionPool(host='zh.wikipedia.org', port=443): Read timed out. (read timeout=10.0)


 87%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                  | 8467/9758 [49:46<07:51,  2.74it/s]

HTTPSConnectionPool(host='zh.wikipedia.org', port=443): Max retries exceeded with url: /w/api.php?action=query&prop=info&titles=%E7%A7%8D_%E5%AD%90&inprop=protection%7Ctalkid%7Cwatched%7Cwatchers%7Cvisitingwatchers%7Cnotificationtimestamp%7Csubjectid%7Curl%7Creadable%7Cpreload%7Cdisplaytitle&format=json&redirects=1 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fc5e4c9ee60>: Failed to establish a new connection: [Errno 101] Network is unreachable'))


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9758/9758 [57:57<00:00,  2.81it/s]


### Info Percentage

In [11]:
data['augmented_sen'] = augmented
temp = data[data['sent']!=data['augmented_sen']]


In [12]:
info_percent = temp.shape[0]/data.shape[0]
print(f"Info Percentage: {info_percent*100:.2f}%")

Info Percentage: 2.25%


### Save Augmented Data

In [13]:
data.to_csv(f'../Dataset/{Set}-wiki-{lang}-{extractor}.csv',index=False)